# Перекрёстная проверка(Кросс-валидация)
Перекрёстная проверка является важной темой в `Машинном обучении`, чтобы гарантировать, что наша модель достаточно надежна. Традиционная стратегия обучения использует 3 части набора данных для обучения, тестирования и проверки.

* обучающий набор — используется для обучения модели и оптимизации гиперпараметров модели
* тестовый набор — используется для оценки того, что модель достаточно обобщает, чтобы правильно работать на данных, на которых она не обучалась. Однако через человека, выполняющего оптимизацию, некоторые знания о тестовом наборе в конечном итоге просачиваются в модель.
* проверочный набор — по этой причине мы используем проверочный набор, который используется в качестве окончательной проверки того, что модель способна обобщать ранее неизвестные данные.

Создание таких наборов ограничивает количество данных, используемых для обучения, и это может снизить способность модели к обучению. Перекрестная проверка позволяет создать надежную модель, разделив обучающие данные на `k` подмножеств. Каждое подмножество использует часть своих данных для обучения, а часть - для тестирования. Следующее подмножество использует другое разделение обучающих и тестовых данных, как вы можете видеть на рисунке ниже. Это более ресурсоемко, но позволяет использовать данные режима для обучения.

!["Cross validation folds"](Cross_Validation.jpg)

Вы также можете быть уверены, что ваша модель способна обобщать, если каждая из групп имеет схожую производительность. Если одна (или несколько) групп действительно достигают результата, а другие работают плохо, вам нужно больше думать о том, как вы разделяете данные. Вы увидите примеры этого ниже.

`Scikit Learn` предлагает множество методов, которые помогают разделить данные на наборы для обучения, тестирования и проверки. Наиболее популярными из тех, которые мы рассмотрим в этом руководстве, являются:

* train_test_split — создает единое разделение на обучающий и тестовый наборы.
* Kfold — создает k-кратные разделения, позволяющие проводить перекрестную проверку
* StratifiedKFold — создает k-кратные разделения, учитывая распределение целевой переменной
* cross_val_score — оценка модели evaluta через перекрестную проверку

Однако следует помнить, что перекрестная проверка подходит не для всех случаев, и следует с умом подходить к вопросу разделения данных.

Подробнее о перекрестной проверке:
* https://scikit-learn.org/stable/modules/cross_validation.html
* https://en.wikipedia.org/wiki/Cross-validation_(statistics)

Давайте рассмотрим несколько реальных вариантов использования. Сначала мы разделим простой **диапазон из 25 чисел**, а затем рассмотрим популярный **набор данных по ирисам**, который использует измерения лепестков и чашелистиков для прогнозирования вида цветка ириса.

In [2]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random

In [3]:
rn = range(1,26)

Начнем с метода Kfold, который разбивает обучающий набор на k-кратные выборки, так что каждый образец используется один раз для тестирования и k-1 раз для обучения.

In [4]:
# чтобы продемонстрировать, как разделяются данные, мы создадим 3 и 5 групп.
# Функция KFold должна быть применена к данным, и она возвращает местоположение (индекс) обучающей и тестовой выборок.
kf5 = KFold(n_splits=5, shuffle=False)
kf3 = KFold(n_splits=3, shuffle=False)

In [ ]:
# Функция Kfold возвращает индексы данных. Наш диапазон от 1 до 25, поэтому индекс равен 0-24
for train_index, test_index in kf3.split(rn):
    print(train_index, test_index)

In [ ]:
# Чтобы получить значения из наших данных, мы используем np.take() для доступа к значению по определенному индексу
for train_index, test_index in kf3.split(rn):
    print(np.take(rn,train_index), np.take(rn,test_index))

## Диаграмма разделения перекрестной проверки

In [9]:
# Давайте разделим наш тестовый диапазон на 5 и 3 части и отобразим разделения на диаграмме.
# Чтобы четко показать, какие данные принадлежат каждому набору, мы сдвинем значения на -.1 и +.1
# первая часть будет содержать значения 0.9 в обучающем наборе и 1.1 в тестовом наборе, вторая 1.9 и 2.1 и т. д.
# мы также дадим каждому набору разный цвет
# поскольку мы повторим это упражнение для перемешанной версии, создадим функцию

def kfoldize(kf, rn, shift=.1):
    train = pd.DataFrame()
    test = pd.DataFrame()
    i = 1
    for train_index, test_index in kf.split(rn):
        train_df = pd.DataFrame(np.take(rn, train_index), columns=["x"])
        train_df["val"] = i - shift
        train = train._append(train_df)

        test_df = pd.DataFrame(np.take(rn, test_index), columns=["x"])
        test_df["val"] = i + shift
        test = test._append(test_df)
        i += 1
    return train, test

In [ ]:
train5, test5 = kfoldize(kf5,rn)
train3, test3 = kfoldize(kf3,rn)

fig,ax = plt.subplots(1,2, figsize=(15,5))
ax[0].scatter(x="x",y="val",c="b",label="train",s=15,data=train5)
ax[0].scatter(x="x",y="val",c="r",label="test",s=15,data=test5)
ax[1].scatter(x="x",y="val",c="b",label="train",s=15,data=train3)
ax[1].scatter(x="x",y="val",c="r",label="test",s=15,data=test3)
ax[0].set_ylabel("Kfold")
ax[0].set_xlabel("feature")
ax[1].set_xlabel("feature")
ax[0].set_title("5 Folds")
ax[1].set_title("3 Folds")
plt.suptitle("Kfold split between train and test features")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
# Давайте удостоверимся, как значения распределены между наборами; Также мы создадим функцию, чтобы мы могли повторять
def kfold_stats(df, name):
    s =  pd.Series({"Min value: ": df["x"].min(),
              "Max value: ": df["x"].max(),
              "Min occurance: ": df["x"].value_counts().min(),
              "Max occurance: ": df["x"].value_counts().max(),
               "Min lenght": df.groupby("val").count().min().values[0],
               "Max lenght": df.groupby("val").count().max().values[0]})
    s.name = name
    return s
pd.concat([kfold_stats(train5, "Train5"), kfold_stats(test5,"Test5"),
          kfold_stats(train3, "Train3"), kfold_stats(test3,"Test3")], 
          axis=1)

В таблице выше приведены некоторые ключевые факты о KFold:

* `Мин. и макс. значение` (Min and Max value) - и обучающий, и тестовый наборы охватывают все признаки
* `Мин. и макс. вхождение` (Min and Max occurrence) - каждое значение встречается один раз в тестовом наборе и k-1 раз в обучающем наборе
* `Длина мин. и макс. значения` (Min and Max value length) - в случае, если у вас есть число признаков, которое не делится на n, некоторые из них будут иметь разное разделение между тестовым и обучающим наборами. Например, в случае 25 признаков и 3 разделений соотношения будут следующими: 16/9, 17/8, 17/8

## Shuffled KFold (Перемешанный KFold)
В предыдущем примере вы видели неперемешанное распределение train/test. Такое распределение может повлиять на результат модели машинного обучения, поэтому часто бывает полезно случайным образом разделить признаки, чтобы доказать возможности вашей модели. Случайное разделение данных достигается параметром `shuffle`. Параметр `random-state` инициирует случайность таким образом, что использование того же случайного состояния обеспечивает то же разделение.

In [12]:
kf42 = KFold(n_splits=5, shuffle=True, random_state=42)
kf123 = KFold(n_splits=5, shuffle=True, random_state=123)

In [ ]:
train42, test42 = kfoldize(kf42,rn)
train123, test123 = kfoldize(kf123,rn)
train123_2, test123_2 = kfoldize(kf123,rn,shift=.25)

fig,ax = plt.subplots(1,2, figsize=(15,5))
ax[0].scatter(x="x",y="val",c="b",label="train",s=15,data=train42) 
ax[0].scatter(x="x",y="val",c="r",label="test",s=15,data=test42)
ax[1].scatter(x="x",y="val",c="b",label="train",s=15,data=train123)
ax[1].scatter(x="x",y="val",c="r",label="test",s=15,data=test123)
ax[1].scatter(x="x",y="val",c="k",label="test second run",s=15,data=test123_2)
ax[0].set_ylabel("Kfold")
ax[0].set_xlabel("feature")
ax[0].set_title("Shuffled KFold with random state 42")
ax[1].set_ylabel("Kfold")
ax[1].set_xlabel("feature")
ax[1].set_title("Shuffled KFold with random state 123")
plt.suptitle("Shuffled KFold")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

## Kfold на реальном наборе данных
Давайте используем Kfold для оценки модели классификации на популярном наборе данных Iris. Он содержит 150 измерений размеров лепестков и чашелистиков 3 разновидностей цветка ириса - setosa, versicolor и virginica. Каждый содержит 50 измерений в наборе.

Набор данных Iris на scikit-learn: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [ ]:
iris = load_iris(return_X_y=False)
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
features = iris['feature_names']
iris_df['target'] = iris.target
iris_df["target_name"] = iris_df['target'].map({i:name for i,name in enumerate(iris.target_names)})
iris_df.sample(5)

In [ ]:
# Давайте посмотрим, сколько образцов каждого типа ирисов есть в нашем наборе.
pd.DataFrame(iris_df.groupby("target_name").size().reset_index()).rename(columns={0:"samples"})

### Single fold (одинарная группа)
Давайте выполним логистическую регрессию с использованием традиционной функции train_test_split, которая разделит данные на обучающий и тестовый наборы так, чтобы каждое целевое значение встречалось как в обучающем, так и в тестовом наборах одинаковое количество раз.

In [16]:
model = LogisticRegression(solver="liblinear", multi_class="auto")

In [17]:
#Логистическая регрессия без Kfold, просто разделенная на 80% обучающего и 20% тестового набора
X = iris_df[features]
y = iris_df["target"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train) 
pd.DataFrame({"Accurancy on Train":[accuracy_score(y_train, model.predict(X_train))],
    "Accurancy on Test":[accuracy_score(y_test, model.predict(X_test))]})

### KFold
Метод Kfold возвращает порядок образцов, выбранных для обучающих и тестовых наборов в каждой свертке. В кадре данных pandas мы используем функцию .iloc для получения правильных строк. Поскольку мы не разделяем данные на X (features) и y (target), также нужно использовать .loc, чтобы выбрать правильные столбцы (.loc[:,features]) или просто выбрать столбец (['target'])

In [ ]:
i = 1
for train_index, test_index in kf3.split(iris_df):
    X_train = iris_df.iloc[train_index].loc[:, features]
    X_test = iris_df.iloc[test_index][features]
    y_train = iris_df.iloc[train_index].loc[:,'target']
    y_test = iris_df.loc[test_index]['target']
        
    model.fit(X_train, y_train)
    print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, model.predict(X_test))}")
    i += 1

Но почему точность в каждой группе **0.0**? Причина в однородности разделений. Поскольку данные упорядочены так, что **setosa** появляется в первых пятидесяти строках набора данных, за которыми следуют **versicolor** и **virginica**, мы добились уникального распределения обучающего набора, что он не содержит никакой цели, ожидаемой в тестовом наборе. Большинство моделей машинного обучения не могут научиться классифицировать класс, который они никогда не видели.

In [ ]:
target_name = iris_df["target"]

fig, ax = plt.subplots(1,3, figsize=(13,3), sharey=True)
for i, (train_index, test_index) in enumerate(kf3.split(iris_df)):
    ax[i].scatter(x=train_index,y=target_name.iloc[train_index],label ="train", c='b')
    ax[i].scatter(x=test_index,y=target_name.iloc[test_index], label = "test", c='r')
    ax[i].set_title(f"Fold {i+1}")

ax[0].set_yticks([0,1,2])
ax[0].set_yticklabels(iris["target_names"])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

## Shuffled KFold
Один из способов обойти эту проблему — использовать перемешанный Kfold.

In [ ]:
dfs = []
kf = KFold(n_splits=3, shuffle=True, random_state=123)
i = 1

for train_index, test_index in kf.split(iris_df):
    X_train = iris_df.iloc[train_index].loc[:, features]
    X_test = iris_df.iloc[test_index].loc[:,features]
    y_train = iris_df.iloc[train_index].loc[:,'target']
    y_test = iris_df.loc[test_index].loc[:,'target']
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, y_pred)}")
    
    s_train = iris_df.iloc[train_index].loc[:,'target_name'].value_counts()
    s_train.name = f"train {i}"
    s_test = iris_df.iloc[test_index].loc[:,'target_name'].value_counts()
    s_test.name = f"test {i}"
    df = pd.concat([s_train, s_test], axis=1, sort=False)
    df["|"] = "|"
    dfs.append(df)
    
    i += 1

In [ ]:
plt.scatter(x=y_train.index,y=iris_df.iloc[train_index].loc[:,'target_name'],label ="train")
plt.scatter(x=y_test.index,y=iris_df.iloc[test_index].loc[:,'target_name'], label = "test")
plt.legend()
plt.show()

You can see that now we pick samples from all three types of irises, however some are chosen more often than others. 

In [ ]:
pd.concat(dfs,axis=1, sort=False)

## Stratified KFold (стратифицированный KFold)

Вместо использования случайного Kfold мы можем использовать StratifiedKFold, которому нужен дополнительный параметр `y`. В качестве `y` вы используете целевую переменную, чтобы Kfold и выбрал сбалансированное распределение целей в каждой группе.

In [ ]:
dfs = []
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)
i = 1
for train_index, test_index in kf.split(iris_df, iris_df["target"]):
    X_train = iris_df.iloc[train_index].loc[:, features]
    X_test = iris_df.iloc[test_index].loc[:,features]
    y_train = iris_df.iloc[train_index].loc[:,'target']
    y_test = iris_df.loc[test_index].loc[:,'target']

    model.fit(X_train, y_train)
    print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, model.predict(X_test))}, doublecheck: {model.score(X_test,y_test)}")
    
    s_train = iris_df.iloc[train_index].loc[:,'target_name'].value_counts()
    s_train.name = f"train {i}"
    s_test = iris_df.iloc[test_index].loc[:,'target_name'].value_counts()
    s_test.name = f"test {i}"
    df = pd.concat([s_train, s_test], axis=1, sort=False)
    df["|"] = "|"
    dfs.append(df)
    
    i += 1

In [ ]:
plt.scatter(x=y_train.index,y=iris_df.iloc[train_index].loc[:,'target_name'],label ="train")
plt.scatter(x=y_test.index,y=iris_df.iloc[test_index].loc[:,'target_name'], label = "test")
plt.legend()
plt.title("Shuffled stratified StratifiedKFold - 3rd Fold")
plt.show()

In [ ]:
pd.concat(dfs,axis=1, sort=False)

## Перекрестная проверка без KFold
Если вы не хотите экспериментировать с разделенными данными, вам не нужно использовать KFolds, cross_validate или cross_val_score проведут обучение, используя ваши данные и предпочитаемое количество разделений, и выдадут вам оценку на тестовом наборе.

In [ ]:
# cross_validate позволяет указать метрики, которые вы хотите видеть
for i, score in enumerate(cross_validate(model, X,y, cv=3)["test_score"]):
    print(f"Accuracy for the fold no. {i} on the test set: {score}")

In [ ]:
for i, score in enumerate(cross_val_score(model, X,y, cv=3)):
    print(f"Accuracy for the fold no. {i} on the test set: {score}")

## Как распределены входные данные

In [32]:
# Чтобы увидеть, как KFold распределяет признаки, давайте взглянем на набор данных Titanic.
# https://www.kaggle.com/c/titanic
titanic = pd.read_csv(r'titanic.csv').sort_values(by="Sex").reset_index(drop=True)
dfs = []
dfs_data = []

kf = StratifiedKFold(n_splits=3, shuffle=False) #, random_state=123
i = 1
for train_index, test_index in kf.split(titanic, titanic["Survived"]):
    X_train = titanic.iloc[train_index].drop(columns=["Survived"])
    X_test = titanic.iloc[test_index].drop(columns=["Survived"])
    y_train = titanic.iloc[train_index].loc[:,"Survived"]
    y_test = titanic.loc[test_index].loc[:,"Survived"]
    
    dfs_data.append({"train": pd.concat([X_train,y_train], axis=1, sort=False), "test": pd.concat([X_test,y_test], axis=1, sort=False)})
    
    #model.fit(X_train, y_train)
    #print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, model.predict(X_test))}, doublecheck: {model.score(X_test,y_test)}")
       
    i += 1

Используя StratifiedKFold, мы ожидаем, что соотношение выживших и жертв будет одинаковым как в обучающем, так и в тестовом наборе данных. Глядя на распределение и графики, это ожидание, похоже, оправдывается.

In [ ]:
statistics = []
for i, data in enumerate(dfs_data):
    for st in ["train","test"]:
        s = data[st][["Survived"]].groupby(["Survived"]).size()
        s.index = s.index.map({0:"Died",1:"Survived"})
        s.name = f"{st} - {i+1}"
        statistics.append(s)

pd.concat(statistics, axis=1).reset_index()

In [ ]:
for i, data in enumerate(dfs_data):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
    
    plot_df_train = dfs_data[i]["train"][["Survived"]].groupby(["Survived"]).size()
    plot_df_train.index = plot_df_train.index.map({0: "Died", 1: "Survived"})
    sns.barplot(x=plot_df_train.index, y=plot_df_train.values, ax=ax[0])
    ax[0].set_title(f"Train set - {i + 1} fold")
    ax[0].set_xlabel('')
    
    plot_df_test = dfs_data[i]["test"][["Survived"]].groupby(["Survived"]).size()
    plot_df_test.index = plot_df_test.index.map({0: "Died", 1: "Survived"})
    sns.barplot(x=plot_df_test.index, y=plot_df_test.values, ax=ax[1])
    ax[1].set_title(f"Test set - {i + 1} fold")
    ax[1].set_xlabel('')
    
    plt.suptitle("Stratification works perfectly for the target variable")
    plt.show()

Однако StratifiedKFold вообще не смотрит на распределение входных признаков. Мы отсортировали входные данные по полу, чтобы продемонстрировать это `.sort_values(by="Sex")`. Как вы можете видеть, соотношение полов в обучающем и тестовом наборах не очень хорошо коррелирует, что может быть проблемой, поскольку у Female гораздо больше шансов, чтобы пережить эту катастрофу. Если ваш тестовый набор содержит в основном мужчин, модель может ожидать гораздо более высокий уровень смертности. Если наоборот, модель может стать очень оптимистичной.

In [ ]:
# таблица, показывающая распределение полов в разделении StratifiedKFold
metric_dfs = []
for i, data in enumerate(dfs_data):
    s = dfs_data[i]["train"].groupby("Sex").size()
    s.name = f"Fold {i+1} Train"
    st = dfs_data[i]["test"].groupby("Sex").size()
    st.name = f"Fold {i+1} Test"
    metric_df = pd.concat([s,st],axis=1)
    metric_df["|"] = "|"
    metric_dfs.append(metric_df)
pd.concat(metric_dfs,axis=1).reset_index()

In [ ]:
for i, data in enumerate(dfs_data):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

    train_plot_df = data["train"]["Sex"].value_counts()
    sns.barplot(x=train_plot_df.index, y=train_plot_df.values, ax=ax[0])
    ax[0].set_title(f"Train set - {i + 1} fold")

    test_plot_df = data["test"]["Sex"].value_counts()
    sns.barplot(x=test_plot_df.index, y=test_plot_df.values, ax=ax[1])
    ax[1].set_title(f"Test set - {i + 1} fold")

    plt.suptitle("Stratification doesn't consider distribution of the features")
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout for suptitle
    plt.show()

Подготовка идеального распределения признаков — довольно сложная задача, особенно если у вас большое количество признаков, и вы хотите обучить и протестировать модель, учитывая все основные комбинации признаков. Мы не можем рассмотреть, как решить балансировку признаков в этой работе, но если бы вы использовали перемешанный StratifiedKFold, распределение было бы лучше. Вы должны знать об этом при построении своей модели, но хорошая новость в том, что перекрестная проверка может помочь вам определить, что что-то не так, если оценка модели значительно отличается в некоторых группах.

## Давайте рассмотрим другие методы перекрестной проверки

### Repeated KFold

In [48]:
from sklearn.model_selection import RepeatedKFold

kf42 = KFold(n_splits=3, shuffle=True, random_state=42)
krf42 = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)

In [ ]:
train42, test42 = kfoldize(kf42,rn)
train123, test123 = kfoldize(krf42,rn)
#train123_2, test123_2 = kfoldize(kf123,rn,shift=.25)

fig,ax = plt.subplots(1,2, figsize=(15,5), sharey=True)
ax[0].scatter(x="x",y="val",c="b",label="train",s=15,data=train42) 
ax[0].scatter(x="x",y="val",c="r",label="test",s=15,data=test42)
ax[1].scatter(x="x",y="val",c="b",label="train",s=15,data=train123)
ax[1].scatter(x="x",y="val",c="r",label="test",s=15,data=test123)
#ax[1].scatter(x="x",y="val",c="k",label="test second run",s=15,data=test123_2)
ax[0].set_ylabel("Kfold")
ax[0].set_xlabel("feature")
ax[0].set_title("Shuffled KFold with random state 42")
ax[1].set_ylabel("Kfold")
ax[1].set_xlabel("feature")
ax[1].set_title("RepeatedKFold with random state 42")
plt.suptitle("Shuffled Repeated KFold")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

### ShuffleSplit
https://scikit-learn.org/stable/modules/cross_validation.html#random-permutations-cross-validation-a-k-a-shuffle-split

In [50]:
from sklearn.model_selection import ShuffleSplit

kf42 = KFold(n_splits=3, shuffle=True, random_state=42)
ss = ShuffleSplit(n_splits=3, test_size=0.5, random_state=42)

In [ ]:
train42, test42 = kfoldize(kf42,rn)
train123, test123 = kfoldize(ss,rn)

fig,ax = plt.subplots(1,2, figsize=(15,5), sharey=True)
ax[0].scatter(x="x",y="val",c="b",label="train",s=15,data=train42) 
ax[0].scatter(x="x",y="val",c="r",label="test",s=15,data=test42)
ax[1].scatter(x="x",y="val",c="b",label="train",s=15,data=train123)
ax[1].scatter(x="x",y="val",c="r",label="test",s=15,data=test123)
ax[0].set_ylabel("Kfold")
ax[0].set_xlabel("feature")
ax[0].set_title("Shuffled KFold with random state 42")
ax[1].set_ylabel("Kfold")
ax[1].set_xlabel("feature")
ax[1].set_title("ShuffleSplit with random state 42 and 50% samples in the test set")
plt.suptitle("Comparison of KFold and ShuffleSplit")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

### Leave One Out

In [52]:
from sklearn.model_selection import LeaveOneOut

kf42 = KFold(n_splits=3, shuffle=True, random_state=42)
loo = LeaveOneOut()

In [ ]:
train42, test42 = kfoldize(kf42,rn)
train123, test123 = kfoldize(loo,rn)

fig,ax = plt.subplots(1,2, figsize=(15,5), sharey=True)
ax[0].scatter(x="x",y="val",c="b",label="train",s=15,data=train42) 
ax[0].scatter(x="x",y="val",c="r",label="test",s=15,data=test42)
ax[1].scatter(x="x",y="val",c="b",label="train",s=15,data=train123)
ax[1].scatter(x="x",y="val",c="r",label="test",s=15,data=test123)
#ax[1].scatter(x="x",y="val",c="k",label="test second run",s=15,data=test123_2)
ax[0].set_ylabel("Kfold")
ax[0].set_xlabel("feature")
ax[0].set_title("Shuffled KFold with random state 42")
ax[1].set_ylabel("Kfold")
ax[1].set_xlabel("feature")
ax[1].set_title("LeaveOneOut on range 1 to 25")
plt.suptitle("Comparison of KFold and LeaveOneOut")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

Leave One Out делает то, что и заявлено, оставляет только одно наблюдение в качестве тестового образца.

Документация Scikit обсуждает эту тему и предлагает: «Как правило, большинство авторов и эмпирические данные предполагают, что 5- или 10-кратная перекрестная проверка должна быть предпочтительнее LOO».
Дополнительная информация: https://scikit-learn.org/stable/modules/cross_validation.html#leave-one-out-loo


### Leave P Out
https://scikit-learn.org/stable/modules/cross_validation.html#leave-p-out-lpo

In [54]:
from sklearn.model_selection import LeavePOut

kf42 = KFold(n_splits=3, shuffle=True, random_state=42)
lpo = LeavePOut(p=2)

In [ ]:
train42, test42 = kfoldize(kf42,rn)
train123, test123 = kfoldize(lpo,rn)

fig,ax = plt.subplots(1,2, figsize=(15,5), sharey=True)
ax[0].scatter(x="x",y="val",c="b",label="train",s=15,data=train42) 
ax[0].scatter(x="x",y="val",c="r",label="test",s=15,data=test42)
ax[1].scatter(x="x",y="val",c="b",label="train",s=15,data=train123)
ax[1].scatter(x="x",y="val",c="r",label="test",s=15,data=test123)
ax[0].set_ylabel("Kfold")
ax[0].set_xlabel("feature")
ax[0].set_title("Shuffled KFold with random state 42")
ax[1].set_ylabel("Kfold")
ax[1].set_xlabel("feature")
ax[1].set_title("LeavePOut with p = 2 on range 1 to 25")
plt.suptitle("Comparison of KFold and LeavePOut")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

Leave P out создает комбинацию ${n \choose p}$, поэтому в нашем примере ${25 \choose 2} = 300$

# Заключение
Мы рассмотрели, как работают методы Sklearn `Kfold`. Мы видим, что они разбивают данные на `n` наборов. Каждый из признаков будет появляться один раз в тестовом наборе и `n-1` раз в обучающем наборе. Вы также можете использовать `ShuffleSplit` или `RepeatedKFold`, если хотите иметь разное распределение между train и test наборами данных.

В обычном KFold каждая группа(fold) будет содержать `1/n` значений в обучающем наборе и `n-1/n` значений в проверочном наборе. Таким образом, для `n=2` 50% будут в тестовом наборе, `n=3` 33%, `n=4` 25% и т. д.

Мы можем либо разбить данные в порядке их появления, используя `shuffle=False`, либо случайным образом, используя `shuffle=True` и опционально указав `random_state`. Наличие одного и того же случайного состояния (random_state) всегда приведет к одному и тому же разделению тестовых и обучающих данных.

Вам не нужно разделять данные вручную, и вы можете применять только методы `cross-validate` или `cross-val-score`. Данные будут разделены на заднем плане, и вы получите окончательную оценку.

Важной концепцией является StratifiedKFold, который гарантирует, что соотношение целевых переменных останется одинаковым в обучающем и тестовом наборах. Это не означает, что соотношение признаков будет одинаковым. Если одно из разделений показывает необычайно низкую или высокую оценку, это означает, что разделение может повлиять на результаты модели.